In [21]:
import pandas as pd
# my app password: wzzpdfszynqgunlu

In [22]:
df = pd.read_excel(r"C:\AdventureWorksTables\Email.xlsx")

In [28]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_emails():
    # Select Excel File
    file_path = filedialog.askopenfilename(title="Select Excel File", filetypes=[("Excel files", "*.xlsx")])
    if not file_path:
        return
    
    sender_email = sender_entry.get()
    password = password_entry.get()
    
    # Read Excel file
    try:
        df = pd.read_excel(file_path)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to read file: {e}")
        return

    # Check required columns
    if 'Email' not in df.columns or 'Name' not in df.columns:
        messagebox.showerror("Error", "Excel file must contain 'Email' and 'Name' columns.")
        return

    smtp_server = "smtp.gmail.com"
    port = 465  # Port for SSL

    context = ssl.create_default_context()
    
    try:
        with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
            server.login(sender_email, password)
            for index, row in df.iterrows():
                recipient_email = str(row['Email']).strip()
                recipient_name = str(row['Name']).strip()

                # Create the email
                message = MIMEMultipart()
                message['From'] = sender_email
                message['To'] = recipient_email
                message['Subject'] = f"Hello {recipient_name}!"
                
                # Body with UTF-8 encoding
                body = f"Dear {recipient_name},\n\nThis is a personalized message."
                message.attach(MIMEText(body, 'plain', 'utf-8'))
                
                # Send the email
                server.send_message(message)
        
        messagebox.showinfo("Success", "Emails sent successfully!")
    
    except smtplib.SMTPAuthenticationError:
        messagebox.showerror("Error", "Authentication failed. Check your email and app password.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# GUI Setup
app = tk.Tk()
app.title("Gmail Email Automation")

tk.Label(app, text="Sender Email:").pack()
sender_entry = tk.Entry(app, width=40)
sender_entry.pack()

tk.Label(app, text="App Password:").pack()
password_entry = tk.Entry(app, show="*", width=40)
password_entry.pack()

send_button = tk.Button(app, text="Send Emails", command=send_emails)
send_button.pack(pady=10)

app.mainloop()
